In [0]:
import string
import re, nltk, spacy, gensim

import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn

import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
tr = str.maketrans("", "", string.punctuation)
stemmer = SnowballStemmer("english")

In [0]:
#load customized stopwords 
stopword_c = pd.read_csv('../Dataset/stopword.csv', encoding = "ISO-8859-1")
stopword_c = stopword[stopword['stopword'] == 1]
stopword_c = stopword['term'].tolist()

# load nltk stopwords
stoplist = stopwords.words('english')
stoplist.extend(stopword)

In [0]:
# load emotional eating posts
ESRE = pd.read_csv('../Dataset/results/lda/post_esre/df_document_topic_post_esre_4.csv')
del ESRE['Unnamed: 0']

In [0]:
ESRE

,Topic0,Topic1,Topic2,Topic3,dominant_topic,id,title,selftext
0,0.00,0.69,0.23,0.07,1,t3_cvhel,Low carb or calorie counting...or both?,"Between April of '09, and April of '10 I lost ..."
1,0.00,0.77,0.22,0.00,1,t3_cvj75,"Hey Reddit, I've lost 70 pounds and I'm still ...",I don't want to post pictures until I get to m...
2,0.01,0.01,0.35,0.64,3,t3_cvnm4,Low-carb breakfast ideas? What does Loseit like?,I've recently started re-working my diet becau...
3,0.01,0.29,0.67,0.03,2,t3_cvqk2,How did you overcome the fear of eating TOO mu...,I'm about 2-2.5 weeks into an attempt at losin...
4,0.01,0.01,0.11,0.88,3,t3_cwk7f,Low carb-ers can eat Chipotle!,"just and fyi to you low carb kiddos, if you go..."
5,0.11,0.82,0.06,0.00,1,t3_cwx49,loseit's biggest loser?,I've been doing this with a group of friends a...
6,0.30,0.47,0.22,0.00,1,t3_cwyxu,The official loseit weight loss event,"Rules: \n\n* Every 1st and 15th, or as close a..."
7,0.01,0.87,0.11,0.01,1,t3_cy4l4,What are the thing you're most looking forward...,"Hey reddit, I'm giving a shot at a new thread...."
8,0.01,0.37,0.43,0.20,2,t3_cy6bb,when do you weigh yourself?,"I seem to fluctuate about 5lbs a day, from mor..."
9,0.08,0.44,0.48,0.00,2,t3_cy6za,The Loose Skin Issue,Anyone who has over 100 or 200lbs to lose will...


In [0]:
# load comments(feedback) on emotional eating posts
comment_name = ['0', '1', '2', '3']

df_comments = pd.DataFrame()
for data in comment_name:
    df = pd.read_csv('../Dataset/loseit_comments/loseit_comments_2005_2018_05_'+data+'.csv', error_bad_lines = False)
    df_comments  = df_comments.append(df)
    print(df_comments.shape)
    comments_shape = df_comments.shape
    comments_nrow = comments_shape[0]
    comments_ncol = comments_shape[1]
    print("datafile added: ", data, " total number of columns: ", comments_ncol, " total number of rows: ", comments_nrow)
df_comments.reset_index(drop=True,inplace=True)
df_comments_esre = df_comments[df_comments['body'].notnull() & df_comments['parent_id'].notnull()]

# df_comments_esre_0 = df_comments_esre[df_comments.link_id.isin(ESRE_id_list)]

(927779, 21)
datafile added:  0  total number of columns:  21  total number of rows:  927779
(1854139, 21)
datafile added:  1  total number of columns:  21  total number of rows:  1854139
(2781830, 21)
datafile added:  2  total number of columns:  21  total number of rows:  2781830
(3709999, 21)
datafile added:  3  total number of columns:  21  total number of rows:  3709999


In [0]:
df_comments_esre = df_comments_esre[(df_comments_esre['body'] != '[removed]') & (df_comments_esre['body'] != '[deleted]')]
df_comments_esre

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,...,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason
0,X gon give it to ya is absolutely Pavlovian co...,False,False,NaN,katied14,"26F | 5'1"" | SW:146 | CW:140.5 | GW:125",NaN,1525348801,t5_2rz8w,t3_8gma16,...,1,1.527028e+09,0,0,dydkwkg,loseit,NaN,NaN,-005lbs,NaN
1,“Why do you ask? Do you think you’re the fathe...,False,False,NaN,nochedetoro,NaN,NaN,1526071952,t5_2rz8w,t3_8ioncb,...,2,1.527359e+09,0,0,dytxi3m,loseit,NaN,NaN,NaN,NaN
2,I can comfortably eat 800 calories a day of mo...,False,False,NaN,rrood23x,,NaN,1525828795,t5_2rz8w,t3_8hwq9z,...,5,1.527247e+09,0,0,dyob2wa,loseit,NaN,NaN,blank,NaN
3,"Exactly! I would buy the combo out of habit, a...",False,False,NaN,Recyart,"M, 5'7"", 214 lb/194 lb/160 lb",NaN,1525628530,t5_2rz8w,t3_8h9b9c,...,2,1.527150e+09,0,0,dyjmf0w,loseit,NaN,NaN,-020lbs,NaN
4,Last week: 245.8\n\nToday: 240.8\n\n\nI had a ...,False,False,NaN,turkeybone,"37M | 5'8"" SW: 275 CW:239 GW: 199",NaN,1525274508,t5_2rz8w,t3_8gew3g,...,2,1.526992e+09,0,0,dybtlvb,loseit,NaN,NaN,-035lbs,NaN
5,NSV: realise my fitness is way up! Endurance i...,False,False,NaN,Ummah_Strong,F\21\SW: 230\ CG: 1720,NaN,1526157182,t5_2rz8w,t3_8iupg7,...,3,1.527392e+09,0,0,dyvl6r5,loseit,NaN,NaN,blank,NaN
6,Thank you for your first post in /r/loseit! Wh...,False,False,NaN,AutoModerator,NaN,NaN,1526670195,t5_2rz8w,t3_8kfmye,...,1,1.527674e+09,0,0,dz794hd,loseit,NaN,moderator,blank,NaN
7,It depends. I know a lot of people who lose th...,False,False,NaN,NotARiver,"F/31/5'5"" l SW:170 CW:141",NaN,1526495912,t5_2rz8w,t3_8jtpxz,...,1,1.527541e+09,0,0,dz368ap,loseit,NaN,NaN,-025lbs,NaN
8,I work it into my calorie budget to split a sm...,False,False,NaN,lineinthesandnolatin,32 F | 5'3 | SW: 190 | CW: 178 | GW:130,NaN,1527294219,t5_2rz8w,t3_8m0kly,...,1,1.527968e+09,0,0,dzl5mbl,loseit,NaN,NaN,-010lbs,NaN
9,This is why i didn't even tell people. I was a...,False,False,NaN,Farahild,"F33 | 5'6"" | SW 141 | GW 130",NaN,1525596930,t5_2rz8w,t3_8h7372,...,3,1.527139e+09,0,0,dyj2cg5,loseit,NaN,NaN,new,NaN


In [0]:
topic_comments = df_comments_esre[df_comments.parent_id.isin(ESRE.id)]


c:\users\hyong\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [0]:
topic_comments

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,...,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason
0,X gon give it to ya is absolutely Pavlovian co...,False,False,NaN,katied14,"26F | 5'1"" | SW:146 | CW:140.5 | GW:125",NaN,1525348801,t5_2rz8w,t3_8gma16,...,1,1.527028e+09,0,0,dydkwkg,loseit,NaN,NaN,-005lbs,NaN
1,“Why do you ask? Do you think you’re the fathe...,False,False,NaN,nochedetoro,NaN,NaN,1526071952,t5_2rz8w,t3_8ioncb,...,2,1.527359e+09,0,0,dytxi3m,loseit,NaN,NaN,NaN,NaN
2,I can comfortably eat 800 calories a day of mo...,False,False,NaN,rrood23x,,NaN,1525828795,t5_2rz8w,t3_8hwq9z,...,5,1.527247e+09,0,0,dyob2wa,loseit,NaN,NaN,blank,NaN
4,Last week: 245.8\n\nToday: 240.8\n\n\nI had a ...,False,False,NaN,turkeybone,"37M | 5'8"" SW: 275 CW:239 GW: 199",NaN,1525274508,t5_2rz8w,t3_8gew3g,...,2,1.526992e+09,0,0,dybtlvb,loseit,NaN,NaN,-035lbs,NaN
5,NSV: realise my fitness is way up! Endurance i...,False,False,NaN,Ummah_Strong,F\21\SW: 230\ CG: 1720,NaN,1526157182,t5_2rz8w,t3_8iupg7,...,3,1.527392e+09,0,0,dyvl6r5,loseit,NaN,NaN,blank,NaN
12,It's annoying. I found that I just use a simil...,False,False,NaN,redditorspaceeditor,"32F 5'7"" CW: 197 GW1: 170",NaN,1527647935,t5_2rz8w,t3_8n276u,...,1,1.528142e+09,0,0,dzsusm9,loseit,NaN,NaN,blank,NaN
16,"Multiple reasons, I think. Firstly, people don...",False,False,NaN,nomnomchikhan,pregnant|SW221|CW217|GW145,NaN,1527607308,t5_2rz8w,t3_8mzhmx,...,207,1.528115e+09,0,0,dzrn1d4,loseit,NaN,NaN,blank,NaN
18,Thank you for your first post in /r/loseit! Wh...,NaN,NaN,NaN,AutoModerator,NaN,NaN,1502650296,t5_2rz8w,t3_6tgxjy,...,2,1.504025e+09,0,0,dlkjqtb,loseit,NaN,moderator,blank,NaN
19,"Nori, furikake should be fine. Cauliflower ric...",False,False,NaN,mr_trantastic,NaN,NaN,1527130453,t5_2rz8w,t3_8lp71d,...,2,1.527890e+09,0,0,dzhe5tq,loseit,NaN,NaN,NaN,NaN
23,Me too! I remember being over 200 at 12 and 8 ...,NaN,NaN,NaN,MrLKK,"M 20 6'2"" | SW ~252 | CW 180",NaN,1503337848,t5_2rz8w,t3_6v2v4g,...,7,1.504256e+09,0,0,dlxktj4,loseit,NaN,NaN,-070lbs,NaN


In [0]:
def preprocess_comments(df):    
    df['unpunct_body'] = df.body.apply(lambda x: x.translate(tr))
    df['tokenized_body'] = df.unpunct_body.apply(lambda x: nltk.word_tokenize(x))
    df['lower_and_tokenized_body'] = df.tokenized_body.apply(lambda x: [y.lower() for y in x])
    df['stemmed_text'] = df.lower_and_tokenized_body.apply(lambda x: [stemmer.stem(y) for y in x])
    df['document'] = df.stemmed_text.map(lambda x: ' '.join([y for y in x]))
    return df
# df_comments_esre = df_comments_esre.drop(['tokenized_body', 'stemmed_text', 'score_hidden', 'archived', 'controversiality', 'gilded'], axis = 1)
# df_comments_esre['document'] = df_comments_esre['body']

In [0]:
# preprocess comments labelled by post topic
topic_comments = preprocess_comments(topic_comments)


In [0]:
def removeStopwords(wordlist, stopwords):
    return [w for w in wordlist if w not in stopwords]
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

nlp = spacy.load('en', disable=['parser', 'ner'])

def preprocess_document(df):
    data_words = list(sent_to_words(df['document'].tolist()))

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # Run in terminal: python3 -m spacy download en

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    data_lemmatized = lemmatization(data_words, allowed_postags=['ADJ', 'ADV', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'VERB'])
    print(data_lemmatized[:2])
    return data_lemmatized


In [0]:
comment_esre = preprocess_document(topic_comments)

['gon give  ya be absolut pavlovian condit  song come on like ok whi be not sweat get  ass gym edit new banger same purpo fit lizzo', 'whi do  ask do  think  re father bonus point  re woman']


In [0]:
def vectorize_document(data_lemmatized):
    vectorizer = CountVectorizer(stop_words=stopwords, min_df = 10, ngram_range =(1, 3))

    data_vectorized = vectorizer.fit_transform(data_lemmatized)
#     data_dense = data_vectorized.todense()
#     print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")
    return vectorizer, data_vectorized

def buildLDA(data_vectorized, i):
    # Build LDA Model
    lda_model = LatentDirichletAllocation(n_components=i,               # Number of topics
                                          max_iter=10,               # Max learning iterations
                                          learning_method='online',   
                                          random_state=100,          # Random state
                                          batch_size=128,            # n docs in each learning iter
                                          evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                          n_jobs = 1,               # Use all available CPUs
                                         )
    lda_output = lda_model.fit_transform(data_vectorized)

    print(lda_model)  # Model attributes
    print("Log Likelihood: ", lda_model.score(data_vectorized))
    print("Perplexity: ", lda_model.perplexity(data_vectorized))
    print(lda_model.get_params())
    return lda_model, lda_output

def get_document_topic(best_lda_model, data_vectorized, data):
    # Create Document - Topic Matrix
    lda_output = best_lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    return df_document_topic

def get_df_topic_distribution(df_document_topic):
    df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
    df_topic_distribution.columns = ['Topic Num', 'Num Documents']
    print(df_topic_distribution)
    return df_topic_distribution

def get_visualization(best_lda_model, data_vectorized, vectorizer, i):
    pyLDAvis.enable_notebook()
    panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
    if(i == 'total'):
        pyLDAvis.save_html(panel, '../dataset/results/lda/comment_esre/lda_comment_esre_' +str(i) + '_' + str(number_of_topics) + '.html')
    else:
        pyLDAvis.save_html(panel, '../dataset/results/lda/comment_esre/lda_comment_esre_topic' +str(i) + '_' + str(number_of_topics) + '.html')
        
    #     print(panel)
    return panel

# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

def get_topic_keywords_matrix(vectorizer, best_lda_model):
    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=100)        

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    print(df_topic_keywords.head())
    return df_topic_keywords

In [0]:
# count-vectorize comments based on term frequency
vectorizer_comment_esre,data_vectorized_comment_esre = vectorize_document(comment_esre)

In [0]:
def lda_model(data_vectorized, number_of_topics, df, i):
    if(i == 'total'):
        lda_model_comment_esre, lda_output_comment_esre = buildLDA(data_vectorized, number_of_topics)
        df_document_topic_comment_esre = get_document_topic(lda_model_comment_esre, data_vectorized, df)
        df_topic_distribution_comment_esre = get_df_topic_distribution(df_document_topic_comment_esre)
        get_visualization(lda_model_comment_esre, data_vectorized, vectorizer_comment_esre, i)
        topic_keyword_matrix_comment_esre = get_topic_keywords_matrix(vectorizer_comment_esre, lda_model_comment_esre)

        
        df_document_topic_comment_esre['body'] = topic_comments.body.tolist()
        df_document_topic_comment_esre['link_id'] = topic_comments.link_id.tolist()
        df_document_topic_comment_esre['id'] = topic_comments.id.tolist()

        # df_document_topic_comment_esre.to_csv('D:/Dropbox/학교/인지과학 협동과정/2018-1/계량심리세미나/기말프로젝트/Dataset/results/lda/comment_esre/df_document_topic_comment_esre_' + str(number_of_topics) + '.csv')
        df_document_topic_comment_esre.to_csv('../Dataset/results/lda/comment_esre/df_document_topic_comment_esre_' +str(i) + '_' + str(number_of_topics) + '.csv')
        df_topic_distribution_comment_esre.to_csv('../Dataset/results/lda/comment_esre/df_topic_distribution_comment_esre_' +str(i) + '_' + str(number_of_topics) + '.csv')
        topic_keyword_matrix_comment_esre.to_csv('../Dataset/results/lda/comment_esre/topic_keyword_matrix_comment_esre_' +str(i) + '_' + str(number_of_topics) + '.csv')
    else:
        lda_model_comment_esre, lda_output_comment_esre = buildLDA(data_vectorized, number_of_topics)
        df_document_topic_comment_esre = get_document_topic(lda_model_comment_esre, data_vectorized, df)
        df_topic_distribution_comment_esre = get_df_topic_distribution(df_document_topic_comment_esre)
        get_visualization(lda_model_comment_esre, data_vectorized, vectorizer_comment_esre[i], i)
        topic_keyword_matrix_comment_esre = get_topic_keywords_matrix(vectorizer_comment_esre[i], lda_model_comment_esre)

        df_document_topic_comment_esre['body'] = topic_comments[i].body.tolist()
        df_document_topic_comment_esre['link_id'] = topic_comments[i].link_id.tolist()
        df_document_topic_comment_esre['id'] = topic_comments[i].id.tolist()
        
        # df_document_topic_comment_esre.to_csv('D:/Dropbox/학교/인지과학 협동과정/2018-1/계량심리세미나/기말프로젝트/Dataset/results/lda/comment_esre/df_document_topic_comment_esre_' + str(number_of_topics) + '.csv')
        df_document_topic_comment_esre.to_csv('../Dataset/results/lda/comment_esre/df_document_topic_comment_esre_topic' +str(i) + '_' + str(number_of_topics) + '.csv')
        df_topic_distribution_comment_esre.to_csv('../Dataset/results/lda/comment_esre/df_topic_distribution_comment_esre_topic' +str(i) + '_' + str(number_of_topics) + '.csv')
        topic_keyword_matrix_comment_esre.to_csv('../Dataset/results/lda/comment_esre/topic_keyword_matrix_comment_esre_topic' +str(i) + '_' + str(number_of_topics) + '.csv')

In [0]:
# build lda model and get topic modeling results for comments on emotional eating post
for number_of_topics in range(4, 7):
    lda_model(data_vectorized_comment_esre, number_of_topics, topic_comments, 'total')

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=4, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)
Log Likelihood:  -718672236.6585522
Perplexity:  19412.482472857202
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 4, 'n_jobs': 1, 'n_topics': None, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}
   Topic Num  Num Documents
0          0         682216
1          2         297517
2          3         235167
3          1          37469


c:\users\hyong\appdata\local\programs\python\python36\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


         Word 0    Word 1 Word 2  Word 3 Word 4   Word 5  Word 6  \
Topic 0     get        go   good  weight   feel     look    lose   
Topic 1    post  question  start    plea  thank  concern  answer   
Topic 2  weight       day   week  calori   lose      eat      lb   
Topic 3     eat    calori   food     day   make       go     get   

               Word 7 Word 8   Word 9   ...   Word 90 Word 91      Word 92  \
Topic 0          work   time     want   ...     littl     ani          bad   
Topic 1  start weight    faq  rloseit   ...     svnsv  includ  daili svnsv   
Topic 2          goal  start     diet   ...      mani    slow        stick   
Topic 3          meal  drink     good   ...      love    tast          big   

        Word 93 Word 94  Word 95 Word 96     Word 97 Word 98 Word 99  
Topic 0   thank     let      put    walk        stop   small   notic  
Topic 1  detail     may   inform    help         day    feat  welcom  
Topic 2    plan     hit      ani   today  calori day  a

c:\users\hyong\appdata\local\programs\python\python36\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


         Word 0    Word 1  Word 2 Word 3 Word 4   Word 5  Word 6  \
Topic 0     get        go  weight   lose  start     time    feel   
Topic 1    post  question   start   plea  thank  concern  answer   
Topic 2  weight    calori    week    eat   lose      day    loss   
Topic 3    look     great    good    get   work      one   peopl   
Topic 4     eat       day  calori   food   make       go     get   

               Word 7  Word 8   Word 9     ...          Word 90      Word 91  \
Topic 0          good      lb     want     ...             veri        point   
Topic 1  start weight     faq  rloseit     ...      daili svnsv  thank share   
Topic 2          diet    bodi      fat     ...              cut        level   
Topic 3           fit  realli    thing     ...              leg       doctor   
Topic 4          meal   drink    today     ...             fast         fill   

        Word 92    Word 93 Word 94         Word 95  Word 96 Word 97 Word 98  \
Topic 0    mile    everyon     

In [0]:
path = '../Dataset/results/lda/comment_esre/df_document_topic_comment_esre_total_6.csv'
df = pd.read_csv(path)
del df['Unnamed: 0']
df['id'] = df['body']

In [0]:
df['body'] = topic_comments.body.tolist()

In [0]:
df.to_csv(path)